In [1]:
import requests
import os
import json
import base64
import pandas as pd
from copy import deepcopy
import math

# API_ENDPOINT_TXS = "https://XXX/cosmos/tx/v1beta1/txs?events=tx.height={}&pagination.offset={}"
# RPC_ENDPOINT = "https://XXX"
BLOCK_TX_FILE_PATH = "data/block_tx/{}.json"
TX_RESULT_LIMIT = 100

UPGRADE_HEIGHT = 4707300
HALT_HEIGHT = 4713064

GAMM_MSG_PREFIX = "/osmosis.gamm.v1beta1"
UNPOOL_WHITELISTED_MSG = "/osmosis.superfluid.MsgUnPoolWhitelistedPool"
EXEC_MSG = "/cosmos.authz.v1beta1.MsgExec"
LOCK_TOKENS_MSG = "/osmosis.lockup.MsgLockTokens"
LOCK_SUPERFLUID_MSG = "/osmosis.superfluid.MsgLockAndSuperfluidDelegate"

## 0. Download block data 

Download all tx responses for every block in [`UPGRADE_HEIGHT`, `HALT_HEIGHT`] to `data/block_tx/`

### Method 1. Get the downloaded data

Downloaded data already avaialble [here](https://fra1.digitaloceanspaces.com/osmosis-halt-data/raw_data/transactions/txs-data.tar.gz)

Extract the archive in `data/block_tx/`


### Method 2. Download data

In [ ]:
for height in range(UPGRADE_HEIGHT, HALT_HEIGHT+1):
    
    print(height, end="\r")
    
    if not os.path.exists(BLOCK_TX_FILE_PATH.format(height)):
        offset = 0
        response = requests.get(API_ENDPOINT_TXS.format(height, offset), headers={"Accept": "application/json"})
        response_json = response.json()
        total = int(response_json["pagination"]["total"])
        tx_responses = response_json["tx_responses"]
        while total > TX_RESULT_LIMIT:
            total -= TX_RESULT_LIMIT
            offset += TX_RESULT_LIMIT
            response = requests.get(API_ENDPOINT_TXS.format(height, offset), headers={"Accept": "application/json"})
            tx_responses.extend(response.json()["tx_responses"])
        with open(BLOCK_TX_FILE_PATH.format(height), "w") as f:
                json.dump(tx_responses, f)

## 1. Load raw txs into pandas DataFrame

### Method 1. Import the downloaded data and save DataFrame to `raw_tx.csv`

In [ ]:
results = []

def filter_txs_data(dict):
    filtered_dict = {key: dict[key] for key in ["height", "txhash", "code", "timestamp", "tx", "logs"]}
    return filtered_dict


for height in range(UPGRADE_HEIGHT, HALT_HEIGHT+1):
    
    print("processing block:", height, "remaining:", "{:5d}".format(HALT_HEIGHT - height), end="\r")
    
    with open(BLOCK_TX_FILE_PATH.format(height)) as f:
        txs_data = json.load(f)

        if txs_data:
            txs = list(filter(filter_txs_data, txs_data)) 
            results += txs

print("\ncreating dataframe...")

raw_df = pd.DataFrame.from_records(results)
raw_df.to_csv("csv/raw_txs.csv", index=False)

print("done.")

### Method 2. Download the Data and load it to a DataFrame

You can download the raw_txs.csv [here](https://fra1.digitaloceanspaces.com/osmosis-halt-data/csv/tx/raw_txs.tar.gz)

Extract it under `csv/raw_txs.csv` 

In [228]:
import ast

raw_df = pd.read_csv("csv/raw_txs.csv")
raw_df.tx = raw_df.tx.apply(ast.literal_eval)
raw_df.logs = raw_df.logs.apply(ast.literal_eval)

## 2. Process txs data

In [285]:
df = raw_df.copy()

In [286]:
# Remove unsuccessful transactions
df = df[df["code"] == 0]
df.reset_index(inplace=True, drop=True)

# Remove unused columns
df.drop(columns=["raw_log", "events", "data", "info", "codespace", "gas_wanted", "gas_used", "code"], inplace=True)

# Expand txs (with logs)
df = df.join(pd.json_normalize(df.tx)[["body.messages"]]).drop(columns=["tx"])
df = df.explode(column=["body.messages", "logs"])
df.reset_index(inplace=True, drop=True)

# Extract events from logs
df = df.join(pd.json_normalize(df.logs)[["events"]]).drop(columns=["logs"])

df.head()

,height,txhash,timestamp,body.messages,events
0,4707300,AF8A1A668EAF07C57365EE2065A27E69FB6550C7300EC5...,2022-06-07T16:24:28Z,{'@type': '/osmosis.gamm.v1beta1.MsgJoinSwapEx...,"[{'type': 'coin_received', 'attributes': [{'ke..."
1,4707300,8FA019013E99D64980674B46F911553BA8E5340FB1B543...,2022-06-07T16:24:28Z,"{'@type': '/cosmos.gov.v1beta1.MsgVote', 'prop...","[{'type': 'message', 'attributes': [{'key': 'a..."
2,4707300,487E8829053D4B7DDE9B038F990486CE332691EBBD41BD...,2022-06-07T16:24:28Z,{'@type': '/osmosis.gamm.v1beta1.MsgJoinSwapEx...,"[{'type': 'coin_received', 'attributes': [{'ke..."
3,4707300,387FD1E9CA5C77908613720B0C004848B6BB38B10FDDD1...,2022-06-07T16:24:28Z,{'@type': '/cosmos.staking.v1beta1.MsgDelegate...,"[{'type': 'coin_received', 'attributes': [{'ke..."
4,4707300,A08E30D44F70ABDB675DC1E68738420CC454D991930AAC...,2022-06-07T16:24:28Z,{'@type': '/ibc.core.client.v1.MsgUpdateClient...,"[{'type': 'message', 'attributes': [{'key': 'a..."


In [287]:
# Expand messages (but execution data will come from events)
df = df.join(pd.json_normalize(df["body.messages"])[["@type", "sender", "grantee", "msgs"]])
df.drop(columns=["body.messages"], inplace=True)

# Filter messages
df = df[(df["@type"].str.contains(GAMM_MSG_PREFIX)) | (df["@type"] == UNPOOL_WHITELISTED_MSG) | (df["@type"] == EXEC_MSG) | (df["@type"] == LOCK_TOKENS_MSG) | (df["@type"] == LOCK_SUPERFLUID_MSG)]
execs_msgs_if_gamm = df[df["@type"] == EXEC_MSG].msgs.apply(lambda c: True if [m for m in c if GAMM_MSG_PREFIX in m["@type"]] else False)
df.drop(execs_msgs_if_gamm[execs_msgs_if_gamm == False].index, axis=0, inplace=True)
df.reset_index(inplace=True, drop=True)

df.head()

,height,txhash,timestamp,events,@type,sender,grantee,msgs
0,4707300,AF8A1A668EAF07C57365EE2065A27E69FB6550C7300EC5...,2022-06-07T16:24:28Z,"[{'type': 'coin_received', 'attributes': [{'ke...",/osmosis.gamm.v1beta1.MsgJoinSwapExternAmountIn,osmo148uzfggn8recx64uwfjfyx5zctsu8xzlcpa5f2,NaN,NaN
1,4707300,487E8829053D4B7DDE9B038F990486CE332691EBBD41BD...,2022-06-07T16:24:28Z,"[{'type': 'coin_received', 'attributes': [{'ke...",/osmosis.gamm.v1beta1.MsgJoinSwapExternAmountIn,osmo144yjwr38rl5zn4hntwad0su3rk9scpz0g5muqr,NaN,NaN
2,4707300,D44A94B87395E34A543BF0FD3602087C5AAC0748B3F747...,2022-06-07T16:24:28Z,"[{'type': 'coin_received', 'attributes': [{'ke...",/osmosis.gamm.v1beta1.MsgSwapExactAmountIn,osmo1zdzn7pfzfryva8nr6s85lctnuavmk2e0utr9ck,NaN,NaN
3,4707301,6B4623822ACF52415F8B7901AA887E2E3601826A5183BB...,2022-06-07T16:24:34Z,"[{'type': 'add_tokens_to_lock', 'attributes': ...",/osmosis.lockup.MsgLockTokens,NaN,NaN,NaN
4,4707301,ACE4297C0E1F18FD28FE027A0CFA910EE75F6311594EBE...,2022-06-07T16:24:34Z,"[{'type': 'add_tokens_to_lock', 'attributes': ...",/osmosis.lockup.MsgLockTokens,NaN,NaN,NaN


In [288]:
# define routine to extract data from events
def process_msg_events(row):
    events = row[3]
    msg_type = row[4]
    sender = row[5]
    if "MsgExec" in msg_type:
        # we have one MsgJoinSwapExternAmountIn in the list of txs done via MsgExec, swap it for the underlying msg
        # we don't need generic code, as there's literally just one MsgExec with one MsgJoinSwapExternAmountIn inside in the entire txs list 
        msg_type = row[7][0]["@type"]
        sender = row[7][0]["sender"]

    if "MsgSwapExactAmountIn" in msg_type or "MsgSwapExactAmountOut" in msg_type:
        for event in events:
            if event["type"] == "token_swapped":
                pool_id_list = list()
                tokens_in_list = list()
                tokens_out_list = list()
                swaps = zip(*(iter(event["attributes"]),) * 5) # 5 entries per single "token_swapped" event
                for swap in swaps:
                    pool_id_list.append(swap[2]["value"])
                    tokens_in_list.append(swap[3]["value"])
                    tokens_out_list.append(swap[4]["value"])
                return msg_type, sender, pool_id_list, tokens_in_list, tokens_out_list
    elif "MsgJoinPool" in msg_type or "MsgJoinSwapExternAmountIn" in msg_type or "MsgJoinSwapShareAmountOut" in msg_type:
        for event in events:
            if event["type"] == "pool_joined":
                pool_id = event["attributes"][2]["value"]
                tokens_in = event["attributes"][3]["value"]
            elif event["type"] == "coinbase":
                shares_minted = event["attributes"][1]["value"]
        return msg_type, sender, pool_id, tokens_in, shares_minted
    elif "MsgExitPool" in msg_type or "MsgExitSwapExternAmountOut" in msg_type or "MsgExitSwapShareAmountIn" in msg_type or "MsgUnPoolWhitelistedPool" in msg_type:
        for event in events:
            if event["type"] == "pool_exited":
                pool_id = event["attributes"][2]["value"]
                tokens_out = event["attributes"][3]["value"]
            elif event["type"] == "burn":
                shares_burned = event["attributes"][1]["value"]
        return msg_type, sender, pool_id, shares_burned, tokens_out
    elif "MsgLockTokens" in msg_type or "MsgLockAndSuperfluidDelegate" in msg_type:
        for event in events:
            if event["type"] == "add_tokens_to_lock" or event["type"] == "lock_tokens":
                token_in = event["attributes"][2]["value"]
                pool_id = token_in.split("/")[-1]
                sender = event["attributes"][1]["value"]
                return msg_type, sender, pool_id, token_in, ""

In [289]:
# extract data from events
df["@type"], df["sender"], df["poolId"], df["tokensIn"], df["tokensOut"] = zip(*df.apply(process_msg_events, axis=1))
df.drop(columns=["events", "grantee", "msgs"], inplace=True)

df.head()

,height,txhash,timestamp,@type,sender,poolId,tokensIn,tokensOut
0,4707300,AF8A1A668EAF07C57365EE2065A27E69FB6550C7300EC5...,2022-06-07T16:24:28Z,/osmosis.gamm.v1beta1.MsgJoinSwapExternAmountIn,osmo148uzfggn8recx64uwfjfyx5zctsu8xzlcpa5f2,722,1018369uosmo,2572033049108385259gamm/pool/722
1,4707300,487E8829053D4B7DDE9B038F990486CE332691EBBD41BD...,2022-06-07T16:24:28Z,/osmosis.gamm.v1beta1.MsgJoinSwapExternAmountIn,osmo144yjwr38rl5zn4hntwad0su3rk9scpz0g5muqr,719,1185077ibc/A0CC0CF735BFB30E730C70019D4218A1244...,18561808163462387644gamm/pool/719
2,4707300,D44A94B87395E34A543BF0FD3602087C5AAC0748B3F747...,2022-06-07T16:24:28Z,/osmosis.gamm.v1beta1.MsgSwapExactAmountIn,osmo1zdzn7pfzfryva8nr6s85lctnuavmk2e0utr9ck,"[641, 678]",[29436000000ibc/67795E528DF67C5606FC20F824EA39...,"[791775171uosmo, 879220465ibc/D189335C6E4A68B5..."
3,4707301,6B4623822ACF52415F8B7901AA887E2E3601826A5183BB...,2022-06-07T16:24:34Z,/osmosis.lockup.MsgLockTokens,osmo19rhv2m3s6vp74yz7csufrsrx4m68lc32wztd4c,1,23891597535214657024gamm/pool/1,
4,4707301,ACE4297C0E1F18FD28FE027A0CFA910EE75F6311594EBE...,2022-06-07T16:24:34Z,/osmosis.lockup.MsgLockTokens,osmo1d65jeqw3kvk6kegp9p2q3t358077062vas2g87,662,4169520195388320gamm/pool/662,


In [290]:
# split swaps into separate rows (one row per poolId)
df = df.explode(column=["poolId", "tokensIn", "tokensOut"])
df.reset_index(inplace=True, drop=True)

df.head()

,height,txhash,timestamp,@type,sender,poolId,tokensIn,tokensOut
0,4707300,AF8A1A668EAF07C57365EE2065A27E69FB6550C7300EC5...,2022-06-07T16:24:28Z,/osmosis.gamm.v1beta1.MsgJoinSwapExternAmountIn,osmo148uzfggn8recx64uwfjfyx5zctsu8xzlcpa5f2,722,1018369uosmo,2572033049108385259gamm/pool/722
1,4707300,487E8829053D4B7DDE9B038F990486CE332691EBBD41BD...,2022-06-07T16:24:28Z,/osmosis.gamm.v1beta1.MsgJoinSwapExternAmountIn,osmo144yjwr38rl5zn4hntwad0su3rk9scpz0g5muqr,719,1185077ibc/A0CC0CF735BFB30E730C70019D4218A1244...,18561808163462387644gamm/pool/719
2,4707300,D44A94B87395E34A543BF0FD3602087C5AAC0748B3F747...,2022-06-07T16:24:28Z,/osmosis.gamm.v1beta1.MsgSwapExactAmountIn,osmo1zdzn7pfzfryva8nr6s85lctnuavmk2e0utr9ck,641,29436000000ibc/67795E528DF67C5606FC20F824EA39A...,791775171uosmo
3,4707300,D44A94B87395E34A543BF0FD3602087C5AAC0748B3F747...,2022-06-07T16:24:28Z,/osmosis.gamm.v1beta1.MsgSwapExactAmountIn,osmo1zdzn7pfzfryva8nr6s85lctnuavmk2e0utr9ck,678,791775171uosmo,879220465ibc/D189335C6E4A68B513C10AB227BF1C1D3...
4,4707301,6B4623822ACF52415F8B7901AA887E2E3601826A5183BB...,2022-06-07T16:24:34Z,/osmosis.lockup.MsgLockTokens,osmo19rhv2m3s6vp74yz7csufrsrx4m68lc32wztd4c,1,23891597535214657024gamm/pool/1,


## 3. Save processed txs as `txs.csv`

In [291]:
df.to_csv("csv/txs.csv", index=None)

At this point the processed DataFrame contains the ordered sequence of `gamm` transactions with the relevant execution data (plus `MsgUnPoolWhitelistedPool` as it removes liquidity from pools, this is marginal and only implemented to perform consistency checks). 
We also include transactions that lock LP shares so we can properly keep track of LP shares illegitimately acquired that get bonded.
 
We can now simulate running all transactions on the various pools, sequentially, starting from the state of pools at `UPGRADE_HEIGHT-1`.

### re-load previously saved `txs.csv`

skip to this part when reloading

In [46]:
df = pd.read_csv("csv/txs.csv")

## 4. Find wallets affected and their credit/debt towards the protocol

In this analysis, the debit accrued due to exploitation will be denominated in all assets exited from the pool(s), not accounting for successive swaps/transfers that finalize the exploit. The attack pattern usually ends up with a final swap that consolidates the gains to a single asset, but this final calculation can be done separately (and is likely already done somewhere by someone, at least for the bigger attackers).

The underlying idea is to simulate a re-run of all transactions that can alter either liquidity or shares in pools, while computing the correct number of shares minted at joins, and liquidity removed at exits. Exits performed with "inflated" shares - i.e. improper shares acquired through the bug -  will build up a liquidity debit towards the specific pool, while clean exits will result in a credit.
We also keep track of inflated shares that went unclaimed, as they should be burned to restore the LP shares health of pools.

[*NOTE*] **do swaps matter? (i.e. are traders on any debt/credit position towards the protocol?)**
The short answer is **no**, since what is inflating is the number of LP shares, so the damage is done only to LP owners (their share get diluted). When someone exits a pool, the pool is exited following the ratio, so liquidity decreases but in a balanced way (so pricing is not impacted) [the selling before sending tokens out of Osmosis via IBC did it though..]. Seemingly, due to how joining is computed (using the wanted LP shares to compute the actual coins added to the pool) the miscalculation is done only when computing the newly minted LP shares, but liquidity is added in a balanced way starting from `shareOutAmount` (see `getMaximalNoSwapLPAmount`).
However, swaps need to be implemented and executed **anyway** in the simulation, since they impact the starting liquidity when joins/exits are made

#### define functions to query node for onchain data

In [2]:
# define functions to query onchain pool and address balance data at various heights
# credit to @george-aj (https://github.com/george-aj/osmosis-nitrogen-extra-gamm-analysis) for the starting point

import codecs
import protos.osmosis.gamm.pool_models.balancer.balancerPool_pb2
import protos.osmosis.gamm.v1beta1.query_pb2 as query_gamms
import protos.cosmos.bank.v1beta1.query_pb2 as query_bank
from google.protobuf.json_format import MessageToDict


def _send_abci_query(request_msg, path, response_msg, height, timeout=5):
    """Encode and send pre-filled protobuf msg to RPC endpoint."""
    # Some queries have no data to pass.
    if request_msg:
        request_msg = codecs.encode(request_msg.SerializeToString(), 'hex')
        request_msg = str(request_msg, 'utf-8')

    req = {
        "jsonrpc": "2.0",
        "id": "1",
        "method": "abci_query",
        "params": {
            "height": str(height),
            "path": path,
            "data": request_msg
        }
    }
    req = json.dumps(req)
    resp = requests.post(RPC_ENDPOINT, req, timeout=timeout).json()
    if 'result' not in resp:
        print(resp)
    response = resp['result']['response']['value']
    if not response:
        return response_msg()
    response = base64.b64decode(response)
    result = response_msg()
    result.ParseFromString(response)
    return result


def get_onchain_pool_data(height):
    request_msg = query_gamms.QueryPoolsRequest()
    request_msg.pagination.limit = 10000
    response_msg = query_gamms.QueryPoolsResponse
    done = False
    while not done:
        try:
            pool_data = _send_abci_query(request_msg=request_msg,
                                         path="/osmosis.gamm.v1beta1.Query/Pools",
                                         response_msg=response_msg,
                                         height=height)
            pool_data = MessageToDict(pool_data)
            done = True
        except Exception as e:
            print(e)
            print("error while fetching pools data for height {}. Retrying...".format(height))

    pool_map = {}
    for pool in pool_data.get('pools'):
        pool_map.update({int(pool.get('id')): pool})

    return pool_map


def get_onchain_balance(height, address, denom):
    request_msg = query_bank.QueryBalanceRequest()
    request_msg.address = address
    request_msg.denom = denom
    response_msg = query_bank.QueryBalanceResponse
    done = False
    while not done:
        try:
            balance = _send_abci_query(request_msg=request_msg,
                                       path="/cosmos.bank.v1beta1.Query/Balance",
                                       response_msg=response_msg,
                                       height=height)
            balance = MessageToDict(balance)
            done = True
        except Exception as e:
            print(e)
            print("error while fetching balance data for address {} at height {}. Retrying...".format(address, height))

    return int(balance["balance"]["amount"])


def get_onchain_balances(height, address):
    request_msg = query_bank.QueryAllBalancesRequest()
    request_msg.address = address
    request_msg.pagination.limit = 1000
    response_msg = query_bank.QueryAllBalancesResponse
    done = False
    while not done:
        try:
            balance = _send_abci_query(request_msg=request_msg,
                                       path="/cosmos.bank.v1beta1.Query/AllBalances",
                                       response_msg=response_msg,
                                       height=height)
            balance = MessageToDict(balance)
            done = True
        except Exception as e:
            print(e)
            print("error while fetching balance data for address {} at height {}. Retrying...".format(address, height))
    return balance["balances"] if "balances" in balance else []

#### define functions to perform join/exit computations

In [3]:
# define functions to compute correct amounts when doing join/exit
# uses `jigu.core` (https://jigu.terra.money/docs/data/sdk.html#decimal-numbers) for a better approximation of `sdk.Dec`
# the logic for join/exit is copied 1~=1 from Osmosis

from jigu.core import Dec
from math import ceil


def pool_balance_of_denom(denom, pool):
    pool_assets = pool["poolAssets"]
    for pool_asset in pool_assets:
        if pool_asset["token"]["denom"] == denom:
            return Dec(pool_asset["token"]["amount"])
    raise Exception("cannot find denom {} in pool {}".format(denom, pool["id"]))


def pool_asset_of_denom(denom, pool):
    pool_assets = pool["poolAssets"]
    for pool_asset in pool_assets:
        if pool_asset["token"]["denom"] == denom:
            return pool_asset
    raise Exception("cannot find denom {} in pool {}".format(denom, pool["id"]))


def pow_int(base, exp):
    if exp == 0:
        return Dec(1)
    tmp = Dec(1)
    i = Dec(exp)
    while i > 1:
        if i % 2 != 0:
            tmp *= base
        i /= 2
        base *= base
    return base * tmp


def abs_diff_with_sign(a, b):
    if a >= b:
        return a - b, False
    else:
        return b - a, True


def pow_approx(base, exp):
    if exp == 0:
        return Dec(1)

    if exp == Dec(0.5):
        smallest_dec = Dec(0.000000000000000001)
        root = Dec(2)
        guess = Dec(1)
        delta = Dec(1)
        iter = 0
        while iter < 100 and delta > smallest_dec:
            prev = pow_int(guess, root - 1)
            if prev == 0:
                prev = smallest_dec
            delta = base / prev
            delta -= guess
            delta /= root

            guess += delta
            iter += 1
        return guess

    pow_precision = Dec(0.00000001)

    x, xneg = abs_diff_with_sign(base, 1)
    base = Dec(x)
    term = Dec(1)
    sum = Dec(1)
    negative = False
    a = Dec(exp)
    i = 0
    while term >= pow_precision:
        c, cneg = abs_diff_with_sign(a, i)
        i += 1
        term *= c * x / i
        if term == 0:
            break
        if xneg:
            negative = not negative
        if cneg:
            negative = not negative
        if negative:
            sum -= term
        else:
            sum += term
    return sum


def pow(base, exp):
    integer = int(exp)
    fractional = exp - integer
    int_pow = pow_int(base, integer)
    if fractional == 0:
        return int_pow
    frac_pow = pow_approx(base, fractional)
    return int_pow * frac_pow


def solve_constant_func_invariant(amt_x_before, amt_x_after, amt_x_weight, amt_y_before, amt_y_weight):
    weight_ratio = Dec(amt_x_weight) / Dec(amt_y_weight)
    y = Dec(amt_x_before) / Dec(amt_x_after)
    return Dec(amt_y_before) * (Dec(1) - pow(y, weight_ratio))


# defines logic to compute the correct amount of shares when performing a join
def correct_join_pool(tokens_in, pool):
    if len(tokens_in) == 0:
        return 0

    # calcJoinSingleAssetTokensIn
    if len(tokens_in) == 1:
        coin = tokens_in[0]
        pool_asset = pool_asset_of_denom(coin["denom"], pool)
        pool_balance = Dec(pool_asset["token"]["amount"])
        normalized_weight = Dec(pool_asset["weight"]) / Dec(pool["totalWeight"])
        swap_fee = pool["poolParams"]["swapFee"]
        swap_fee = Dec("0." + "0" * (18 - len(swap_fee)) + swap_fee)
        token_amt_after_fee = Dec(coin["amount"]) * (Dec(1) - (Dec(1) - normalized_weight) * swap_fee)
        num_shares = -solve_constant_func_invariant(pool_balance + token_amt_after_fee, pool_balance, normalized_weight, pool["totalShares"]["amount"], 1)
        return int(num_shares)

    # MaximalExactRatioJoin
    min_share_ratio = Dec(10**100)
    max_share_ratio = Dec(0)
    coin_share_ratios = dict()
    for coin in tokens_in:
        amt_in = Dec(coin["amount"])
        share_ratio = amt_in / pool_balance_of_denom(coin["denom"], pool)
        if share_ratio < min_share_ratio:
            min_share_ratio = share_ratio
        if share_ratio > max_share_ratio:
            max_share_ratio = share_ratio
        coin_share_ratios[coin["denom"]] = share_ratio
    num_shares = int((min_share_ratio * Dec(pool["totalShares"]["amount"])))
    rem_coins = list()
    added_coins = list()
    if min_share_ratio != max_share_ratio:
        for coin in tokens_in:
            if coin_share_ratios[coin["denom"]] == min_share_ratio:
                added_coins.append(coin)
                continue
            used_amt = ceil(min_share_ratio * pool_balance_of_denom(coin["denom"], pool))
            new_amt = int(coin["amount"]) - int(used_amt)
            added_coins.append({"denom": coin["denom"], "amount": used_amt})
            if new_amt > 0:
                rem_coins.append({"denom": coin["denom"], "amount": new_amt})

    updated_pool = deepcopy(pool)
    updated_pool["totalShares"]["amount"] = int(updated_pool["totalShares"]["amount"]) +  num_shares
    for coin in added_coins:
        for pool_asset in updated_pool["poolAssets"]:
            if pool_asset["token"]["denom"] == coin["denom"]:
                pool_asset["token"]["amount"] = int(pool_asset["token"]["amount"]) + int(coin["amount"])

    # calcJoinSingleAssetTokensIn
    for coin in rem_coins:
        pool_asset = pool_asset_of_denom(coin["denom"], updated_pool)
        pool_balance = Dec(pool_asset["token"]["amount"])
        normalized_weight = Dec(pool_asset["weight"]) / Dec(pool["totalWeight"])
        swap_fee = pool["poolParams"]["swapFee"]
        swap_fee = Dec("0." + "0" * (18 - len(swap_fee)) + swap_fee)
        token_amt_after_fee = Dec(coin["amount"]) * (Dec(1) - (Dec(1) - normalized_weight) * swap_fee)
        new_shares = -solve_constant_func_invariant(pool_balance + token_amt_after_fee, pool_balance, normalized_weight, updated_pool["totalShares"]["amount"], 1)
        num_shares += int(new_shares)

    return num_shares


# compute liquidity removed as a result of burning the provided amount of shares. Used to compute shares on corrected pool history.
def burn_shares(pool, shares_burned_amt):
    shares_burned_amt = Dec(shares_burned_amt)
    exit_fee = pool["poolParams"]["exitFee"]
    exit_fee = Dec("0." + "0" * (18 - len(exit_fee)) + exit_fee)
    if exit_fee > 0:
        shares = shares_burned_amt * (Dec(1) - exit_fee)
    else:
        shares = shares_burned_amt
    share_out_ratio = shares / Dec(pool["totalShares"]["amount"])
    tokens_out = list()
    for pool_asset in pool["poolAssets"]:
        amt_out = int(share_out_ratio * Dec(pool_asset["token"]["amount"]))
        tokens_out.append({"denom": pool_asset["token"]["denom"], "amount": amt_out}) 
    return tokens_out

#### define helper functions to handle coins and pool assets

In [4]:
# quick and dirty function to split a normalized coin string into denom and amount
def parse_token_str(token_str):
    denom_index_start = token_str.find(next(filter(str.isalpha, token_str)))
    amount, denom = int(token_str[:denom_index_start]), token_str[denom_index_start:]
    return {"amount": amount, "denom": denom}


# another quick and dirty function to perform (sort of) `sdk.Coins.Sub()`
def sub_coins(a, b):
    res = deepcopy(a)
    for coinb in b:
        for coina in res:
            if coina["denom"] == coinb["denom"]:
                if int(coina["amount"]) < int(coinb["amount"]):
                    raise ArithmeticError("would result in coin with negative amount -- {} -- {}".format(a, b))
                amt = int(coina["amount"]) - int(coinb["amount"])
                # amt = 0 if amt < 0 else amt # clamp (< tolerance) negative values to 0
                coina["amount"] = amt
                break
        else:
            raise Exception("Unable to find denom {} in primary coin list".format(coinb["denom"]))
    return [c for c in res if c["amount"] > 0]

# another quick and dirty function to perform (sort of) `sdk.Coins.Add()`
def add_coins(a, b):
    res = deepcopy(a)
    for coinb in b:
        for coina in res:
            if coina["denom"] == coinb["denom"]:
                amt = int(coina["amount"]) + int(coinb["amount"])
                coina["amount"] = amt
                break
        else:
            # coinb not found in coina, so add it to the list as is
            res.append(deepcopy(coinb))
    return res


# update the pool assets using the provided lists of tokens in and tokens out
def update_pool_assets(tokens_in, tokens_out, pool):
    pool_assets = pool["poolAssets"]
    for token_in in tokens_in:
        for pool_asset in pool_assets:
            if pool_asset["token"]["denom"] == token_in["denom"]:
               pool_asset["token"]["amount"] = str(int(pool_asset["token"]["amount"]) + int(token_in["amount"]))
               break
        else:
            raise Exception("cannot find asset {} in pool {}".format(token_in["denom"], pool["id"]))
    for token_out in tokens_out:
        for pool_asset in pool_assets:
            if pool_asset["token"]["denom"] == token_out["denom"]:
               pool_asset["token"]["amount"] = str(int(pool_asset["token"]["amount"]) - int(token_out["amount"]))
               break
        else:
            raise Exception("cannot find asset {} in pool {}".format(token_out["denom"], pool["id"]))

#### define main txs processing logic

In [40]:
def handle_swap(tx, proper_pools, corrupted_pools, affected_wallets):
    pool_id = int(tx.poolId)
    # just update pool balances using execution data
    # we chose not to exclude swaps that happen with stolen liquidity since
    # it would alter pricing over time. The model would as a consequence complicate 
    # a lot and keeping consistency might become unmanageable
    token_in = parse_token_str(tx["tokensIn"]) # can only be one token
    token_out = parse_token_str(tx["tokensOut"]) # can only be one token
    update_pool_assets([token_in], [token_out], proper_pools[pool_id])
    update_pool_assets([token_in], [token_out], corrupted_pools[pool_id])

In [41]:
def handle_join(tx, proper_pools, corrupted_pools, affected_wallets):
    addr = tx.sender
    pool_id = int(tx.poolId)
    proper_tokens_in = tokens_in = [parse_token_str(t) for t in tx.tokensIn.split(",")]
    shares_added = parse_token_str(tx.tokensOut)
    if proper_pools[pool_id]["totalShares"]["denom"] != shares_added["denom"]:
        raise Exception("invalid shares minted for tx type {} hash {} height".format(tx["@type"], tx["txhash"], tx["height"]))
    # update `proper_pool` data with proper shares amount, and use execution data to update `corrupted_pool`
    corrupted_pools[pool_id]["totalShares"]["amount"] = str(int(corrupted_pools[pool_id]["totalShares"]["amount"]) + int(shares_added["amount"]))

    if addr in affected_wallets and pool_id in affected_wallets[addr] and "tokens_stolen" in affected_wallets[addr][pool_id] and affected_wallets[addr][pool_id]["tokens_stolen"]:
        # if account previously stole assets from this pool and is adding it back
        # consider all shares produced by joininig with stolen assets as "inflated" (i.e. acquired through the exploit)
        # NOTE: stolen liquidity moving across pools is not tracked as it would overcomplicate the model with little benefit
        actual_tokens_in = deepcopy(tokens_in)
        pool_stolen = affected_wallets[addr][pool_id]["tokens_stolen"]
        for coin_stolen in pool_stolen:
            for coin_in in actual_tokens_in:
                if coin_stolen["denom"] == coin_in["denom"]:
                    if coin_stolen["amount"] >= coin_in["amount"]:
                        coin_in["amount"], coin_stolen["amount"] = 0, coin_stolen["amount"] - coin_in["amount"]
                    else:
                        coin_in["amount"], coin_stolen["amount"] = coin_in["amount"] - coin_stolen["amount"], 0
                    break
        affected_wallets[addr][pool_id]["tokens_stolen"] = [t for t in affected_wallets[addr][pool_id]["tokens_stolen"] if t["amount"] > 0]
        proper_tokens_in = [t for t in actual_tokens_in if t["amount"] > 0]
        if len(proper_tokens_in) > 1 and len(proper_tokens_in) != len(proper_pools[pool_id]["poolAssets"]):
            raise Exception("unsupported join!")
    
    correct_shares_amt = correct_join_pool(proper_tokens_in, proper_pools[pool_id])
    proper_pools[pool_id]["totalShares"]["amount"] = str(int(proper_pools[pool_id]["totalShares"]["amount"]) + int(correct_shares_amt))
    update_pool_assets(proper_tokens_in, [], proper_pools[pool_id])
    update_pool_assets(tokens_in, [], corrupted_pools[pool_id])
    # add entry for wallet-pool to track creation of inflated shares
    if addr not in affected_wallets:
        affected_wallets[addr] = {pool_id: {"shares_inflated": 0}}
    elif pool_id not in affected_wallets[addr]:
        affected_wallets[addr][pool_id] = {"shares_inflated": 0}
    elif "shares_inflated" not in affected_wallets[addr][pool_id]:
        affected_wallets[addr][pool_id]["shares_inflated"] = 0
    # a negative amount here means the accound should have actually received more shares
    affected_wallets[addr][pool_id]["shares_inflated"] += shares_added["amount"] - correct_shares_amt

In [42]:
def handle_lock(tx, affected_wallets):
    tx_height = int(tx.height)
    addr = tx.sender
    pool_id = int(tx.poolId)
    if addr in affected_wallets and pool_id in affected_wallets[addr] and "shares_inflated" in affected_wallets[addr][pool_id]:
        shares_inflated_amt = affected_wallets[addr][pool_id]["shares_inflated"]
        shares_locked = parse_token_str(tx.tokensIn)
        shares_balance_amt = get_onchain_balance(tx_height-1, addr, shares_locked["denom"])
        shares_valid_amt = shares_balance_amt - shares_inflated_amt
        if shares_locked["amount"] > shares_balance_amt:
            raise Exception("this should not happen")
        if shares_locked["amount"] > shares_valid_amt:
            diff = shares_locked["amount"] - shares_valid_amt
            affected_wallets[addr][pool_id]["shares_inflated"] = shares_inflated_amt - diff
            if "shares_inflated_locked" not in affected_wallets[addr][pool_id]:
                affected_wallets[addr][pool_id]["shares_inflated_locked"] = 0
            affected_wallets[addr][pool_id]["shares_inflated_locked"] += diff

In [43]:
def handle_exit(tx, proper_pools, corrupted_pools, affected_wallets):    
    # 3 different cases:
    #   1. Account is burning valid shares and did not perform a join post v9 upgrade (no inflated shares owned)
    #      In this case, the account is due a refund computed using the `proper_pools` state
    #   2. Account is burning a number of shares that would be entirely covered by the valid shares possessed.
    #      The account has a number of inflates shares in its possession, but they are not claimed in this tx
    #      (and could be just returned and burned if left unclaimed)
    #      Also in this case, the account is due a refund computed using the `proper_pools` state.
    #      If the wallet claims the inflated shares in the future, the net will be a debt and not a credit towards the pool 
    #   3. Account is burning inflated shares (possibly on top of valid shares)
    #      The account is extracting liquidity using the exploit, we compute this amount and track it as a debt
    #      towards the pool
    tx_height = int(tx.height)
    addr = tx.sender
    pool_id = int(tx.poolId)
    tokens_out = [parse_token_str(t) for t in tx["tokensOut"].split(",")]
    shares_burned = parse_token_str(tx["tokensIn"])
    if proper_pools[pool_id]["totalShares"]["denom"] != shares_burned["denom"]:
        raise Exception("invalid shares minted for tx type {} hash {} height".format(tx["@type"], tx["txhash"], tx_height))

    if addr in affected_wallets and pool_id in affected_wallets[addr] and "shares_inflated" in affected_wallets[addr][pool_id]:
        only_valid_shares = False
        # if entry is in `affected_wallets`, the account performed a join earlier
        shares_inflated_amt = affected_wallets[addr][pool_id]["shares_inflated"]
        shares_balance_amt = get_onchain_balance(tx_height-1, addr, shares_burned["denom"])
        proper_shares_burned_amt = shares_valid_amt = max(0, shares_balance_amt - shares_inflated_amt)
        if shares_burned["amount"] > shares_balance_amt:
            raise Exception("this should not happen")

        if shares_burned["amount"] > shares_valid_amt:
            affected_wallets[addr][pool_id]["shares_inflated"] = shares_inflated_amt - (shares_burned["amount"] - shares_valid_amt)
        else:
            # in case the shares burned are less or equal than the valid shares available, use those first 
            # (if inflated shares are never claimed, they can simply be burned)
            only_valid_shares = True
            proper_shares_burned_amt = shares_burned["amount"]

        # simulate burning correct number of shares
        correct_tokens_out = burn_shares(proper_pools[pool_id], proper_shares_burned_amt)

        if only_valid_shares:
            # the tx is using only valid shares, so we will compute a refund.
            # If the wallet claims the inflated shares in the future, the net will be a debt and not a credit towards the pool
            diff = sub_coins(correct_tokens_out, tokens_out)
            if diff:
                if "tokens_owed" not in affected_wallets[addr][pool_id]:
                    affected_wallets[addr][pool_id]["tokens_owed"] = diff
                else:
                    affected_wallets[addr][pool_id]["tokens_owed"] = add_coins(affected_wallets[addr][pool_id]["tokens_owed"], diff) 
        else:
            # add entry for stolen amount
            try:
                diff = sub_coins(tokens_out, correct_tokens_out)
                if "tokens_stolen" not in affected_wallets[addr][pool_id]:
                    affected_wallets[addr][pool_id]["tokens_stolen"] = diff
                else:
                    affected_wallets[addr][pool_id]["tokens_stolen"] = add_coins(affected_wallets[addr][pool_id]["tokens_stolen"], diff)

            except Exception as e:
                # negative amounts? print out exception, so we can investigate
                if type(e) is not ArithmeticError:
                    print("found exception for tx: {} [{}]".format(tx.txhash, proper_shares_burned_amt))
                    raise e
                else:
                    try:
                        print("[warn] discount {} (due to approximations) in tx: {}".format(sub_coins(correct_tokens_out, tokens_out), tx.txhash))
                    except:
                        print("[warn] amount {} discounted to {} (due to approximations) in tx: {}".format(correct_tokens_out, tokens_out, tx.txhash))
                    correct_tokens_out = tokens_out
    else:
        # this exit does not happen after a v9 join, so the account is possibly due a refund
        proper_shares_burned_amt = shares_burned["amount"]
        correct_tokens_out = burn_shares(proper_pools[pool_id], proper_shares_burned_amt)
        diff = sub_coins(correct_tokens_out, tokens_out)
        if diff:
            if addr not in affected_wallets:
                affected_wallets[addr] = {pool_id: {}}
            elif pool_id not in affected_wallets[addr]:
                affected_wallets[addr][pool_id] = {}
            if "tokens_owed" not in affected_wallets[addr][pool_id]:
                affected_wallets[addr][pool_id]["tokens_owed"] = diff
            else:
                affected_wallets[addr][pool_id]["tokens_owed"] = add_coins(affected_wallets[addr][pool_id]["tokens_owed"], diff)
    
    # update `proper_pools` with valid data
    update_pool_assets([], correct_tokens_out, proper_pools[pool_id])
    proper_pools[pool_id]["totalShares"]["amount"] = str(int(proper_pools[pool_id]["totalShares"]["amount"]) - int(proper_shares_burned_amt))
    # update `corrupted_pools` with execution data
    update_pool_assets([], tokens_out, corrupted_pools[pool_id])
    corrupted_pools[pool_id]["totalShares"]["amount"] = str(int(corrupted_pools[pool_id]["totalShares"]["amount"]) - int(shares_burned["amount"]))
    

In [44]:
# apply a tx on both "proper" and "corrupted" pools. The "corrupted pools" data only use real block data (also for joins) and is used for consistency checks with onchain data
def apply_tx(tx, proper_pools, corrupted_pools, affected_wallets):
    msg_type = tx["@type"]
    
    if "MsgSwapExactAmountIn" in msg_type or "MsgSwapExactAmountOut" in msg_type:
        handle_swap(tx, proper_pools, corrupted_pools, affected_wallets)

    elif "MsgExitSwapExternAmountOut" in msg_type or "MsgExitSwapShareAmountIn" in msg_type:
        raise Exception("not implemented - should not be needed")

    elif "MsgExitPool" in msg_type or "MsgUnPoolWhitelistedPool" in msg_type:
        handle_exit(tx, proper_pools, corrupted_pools, affected_wallets)

    elif "MsgJoinPool" in msg_type or "MsgJoinSwapExternAmountIn" in msg_type or "MsgJoinSwapShareAmountOut" in msg_type:
        handle_join(tx, proper_pools, corrupted_pools, affected_wallets)

    elif "MsgLockTokens" in msg_type or "MsgLockAndSuperfluidDelegate" in msg_type:
        handle_lock(tx, affected_wallets)

### run analysis

this is done sequentially, row-by-row. It can take a while...

In [47]:
# query initial pools data (before upgrade takes effect)
initial_pools_data = get_onchain_pool_data(UPGRADE_HEIGHT-1)
# update swap fees on pool 1 (happened at update time)
initial_pools_data[1]["poolParams"]["swapFee"] = "2000000000000000"

corrected_pools_history = list()
affected_wallets = dict()
proper_pools = initial_pools_data
corrupted_pools = deepcopy(initial_pools_data) # this is to compare with onchain data for consistency
## print("processing block:", UPGRADE_HEIGHT, "remaining:", "{:5d}".format(HALT_HEIGHT - UPGRADE_HEIGHT), end="\r")
last_index = len(df) - 1
for index, tx in df.iterrows():
    # apply tx to update data structures
    apply_tx(tx, proper_pools, corrupted_pools, affected_wallets)
    # get netx tx height
    curr_height = int(tx["height"])
    if index == last_index:
        print("\ndone")
        ## next_height = curr_height + 1
    ## else:
    ##     next_height = int(df.loc[index+1]["height"])
    ## if curr_height < next_height: # disabled to speedup computation
        # next tx is in a new block, finalize pools for this height
        ## if index != last_index:
        ##     print("processing block:", next_height, "remaining:", "{:5d}".format(HALT_HEIGHT - next_height), end="\r")
        ## corrected_pools_history.append(deepcopy(proper_pools))
        # perform sanity check of "corrupted" pools state against onchain data
        ## onchain_pools_data = get_onchain_pool_data(curr_height)
        ## if onchain_pools_data != corrupted_pools:
        ##     raise Exception("inconsistency found between simulated and onchain data for pools at height {}".format(curr_height))

[warn] discount [{'denom': 'ibc/D189335C6E4A68B513C10AB227BF1C1D38C746766278BA3EEB4FB14124F1D858', 'amount': 352}, {'denom': 'uosmo', 'amount': 311}] (due to approximations) in tx: 848C1E87C13D509CC6603EAA739F14719C22D403814B9204716A7517B3AC71DE
[warn] discount [{'denom': 'uosmo', 'amount': 635}] (due to approximations) in tx: E64605C9D726F20354160BACB9796C4B6AEE4C2188D4CA40C502D4484E3C5E4C
[warn] discount [{'denom': 'ibc/0954E1C28EB7AF5B72D24F3BC2B47BBB2FDF91BDDFD57B74B99E133AED40972A', 'amount': 14}, {'denom': 'uosmo', 'amount': 16}] (due to approximations) in tx: C61ECF7A3D299202EEE25BD6E32030E6D547AE702B39D7D4ABB0116BAB616D9F
[warn] discount [{'denom': 'ibc/6AE98883D4D5D5FF9E50D7130F1305DA2FFA0C652D1DD9C123657C6B4EB2DF8A', 'amount': 2746395}] (due to approximations) in tx: E6355A4B7DAC1DF2DE329CDC1E10843A9833273F07C07C2099F95622B51FC6B3
[warn] discount [{'denom': 'ibc/6AE98883D4D5D5FF9E50D7130F1305DA2FFA0C652D1DD9C123657C6B4EB2DF8A', 'amount': 1368663}] (due to approximations) in t

## 5. Process analysis retults

In [5]:
def token_to_str(tokens_list):
    if type(tokens_list) is not list:
        return ""
    return ",".join([str(token["amount"]) + token["denom"] for token in tokens_list])

def net_difference_tokens(row):
    stolen = row[4]
    owed = row[5]
    if type(stolen) is list and stolen and type(owed) is list and owed:
        # we assume stolen > owed
        try:
            stolen = sub_coins(stolen, owed)
            owed = ""
        except Exception as e:
            print("tokens stolen < owed for address {} on pool {}. please verify manually".format(row[0], row[1]))
    return stolen, owed

In [48]:
# generate pandas DataFrame
addrpool_df = pd.DataFrame.from_dict([[addr, pool_id, pool_data] for addr, addr_pools in affected_wallets.items() for pool_id, pool_data in addr_pools.items() if type(pool_data) is dict])
addrpool_df.columns = ["address", "pool_id", "pool_data"]

# expand pool data
addrpool_df = addrpool_df.join(pd.json_normalize(addrpool_df["pool_data"])[["shares_inflated", "shares_inflated_locked", "tokens_stolen", "tokens_owed"]]).drop(columns=["pool_data"])

addrpool_df.head()

,address,pool_id,shares_inflated,shares_inflated_locked,tokens_stolen,tokens_owed
0,osmo148uzfggn8recx64uwfjfyx5zctsu8xzlcpa5f2,722,0,NaN,NaN,NaN
1,osmo144yjwr38rl5zn4hntwad0su3rk9scpz0g5muqr,719,0,18348243256990,NaN,NaN
2,osmo144yjwr38rl5zn4hntwad0su3rk9scpz0g5muqr,601,0,100125363017,NaN,NaN
3,osmo144yjwr38rl5zn4hntwad0su3rk9scpz0g5muqr,604,0,621298171,NaN,NaN
4,osmo144yjwr38rl5zn4hntwad0su3rk9scpz0g5muqr,648,0,23031307428802,NaN,NaN


In [49]:
# compute net difference if entries in both `tokens_stolen` and `tokens_owned`
addrpool_df["tokens_stolen"], addrpool_df["tokens_owed"] = zip(*addrpool_df.apply(net_difference_tokens, axis=1))

# convert tokens list into strings
addrpool_df.tokens_owed = addrpool_df.tokens_owed.apply(token_to_str)
addrpool_df.tokens_stolen = addrpool_df.tokens_stolen.apply(token_to_str)

addrpool_df.sort_values(by=["pool_id", "address"], ascending=True, inplace=True)

addrpool_df

,address,pool_id,shares_inflated,shares_inflated_locked,tokens_stolen,tokens_owed
1636,osmo1060t4vlql9ngh4jj4h67nsyu4lmljyd8pesy5q,1,0,6337397659875570553,,
1319,osmo10678cw3ts8xlt5gftju2gagz7mfkrs052j7a00,1,0,23136296384054444,,
1303,osmo10amfgz727decdtg0ues0twf0ngf8qvscnrflsh,1,217635871083225,NaN,,
2631,osmo10hfp70p9nucq7grtmasskyut7qt9ezp0645h02,1,NaN,NaN,,708ibc/27394FB092D2ECCD56123C74F36E4C1F926001C...
2563,osmo10ppp525dl5fxmvwxrtkds40vzkzchrxnemg858,1,0,2190166987223601,,
...,...,...,...,...,...,...
1531,osmo14wnlg7xchkfhd60fk4g9989rg5h050q8n3sp65,730,2040943989901491158,NaN,,
691,osmo16jexsf558vvdwwl2cfj0mzr4x63xjpac5cs9rv,732,-44531,NaN,,
2510,osmo1mll3rvz07vds9tez3370dvespzjzljlf22eqr7,732,103,NaN,,
1226,osmo1uvnk984yhpw48jfu5srvsrqdt03kkvlcjqx8x5,733,-3703000,NaN,,


## 6. Save wallets info as `wallets-pools.csv`

The dataframe contains an entry for each tuple `(address, pool_id)` (an address may have liabilities towards more than one pool) with the following amounts:

- `shares_inflated` represents the amount of LP shares obtained through the exploit that should be burned. In case the amount is negative, the address should have actually received more LP shares
- `shares_inflated_locked` represents the amount of `shares_inflated` that are bonded
- `tokens_stolen` represents the amount of assets that the address has extracted using the exploit. This represents the amount of tokens stolen that should be returned to the pool liquidity.   
- `tokens_owed` represents the amount of tokens that the address should have received but did not receive due to the exploit. This amount should be airdropped to the address.

In [50]:
addrpool_df.to_csv("csv/wallets-pools.csv", index=None)

## 7. Rektdrop calculation

We use `proper_pools` and `corrupted_pools` as previously computed (i.e. effectively at `HALT_HEIGHT`) along with `affected_wallets` to find the amount that LP providers should be airdropped to compensate for stolen funds.

While computing airdrop amounts, we also keep into account LP shares illegitimately owned (`shares_inflated` or `shares_inflated_locked`) and appropriately penalize accounts that have those (or stole liquidity while the exploit was available).

We do this only for pools that were affected by the exploit in a considerable manner: `1`, `678`, `704`, `712`.

There is a small number of other pools affected for which the amount stolen is negligible (less than 20 OSMO aggregate value in all cases) which are intentionally left out.

Moreover, pool `722` had around ~800 OSMO aggregate value stolen, and is also not included in the analysis.

The only other serious attack was on pool `561` by address `osmo1jfxcl8ja3nnfjduqemptknz2j6nk6502zp3rte`, which at `HALT-HEIGHT` has the vast majority of stolen funds as LP shares in the pool, and has since then exited and returned the stolen OSMO (~8450 OSMO). Analysis for `561` is left out and can be treated separately if needed.

### define some functions to query node for onchain for data

In [70]:
# assumes you have run cell with same title in sec. 4

import protos.cosmos.auth.v1beta1.auth_pb2
import protos.cosmos.crypto.secp256k1.keys_pb2
import protos.cosmos.crypto.multisig.keys_pb2
import protos.cosmos.vesting.v1beta1.vesting_pb2
import protos.cosmos.auth.v1beta1.query_pb2 as query_auth
import protos.osmosis.lockup.query_pb2 as query_lockup


def get_accounts(height):
    request_msg = query_auth.QueryAccountsRequest()
    request_msg.pagination.limit = 10000
    response_msg = query_auth.QueryAccountsResponse
    accounts = []
    next = True
    while next:
        done = False
        while not done:
            try:
                res = _send_abci_query(request_msg=request_msg,
                                       path="/cosmos.auth.v1beta1.Query/Accounts",
                                       response_msg=response_msg,
                                       height=height,
                                       timeout=10)
                done = True
            except Exception as e:
                print(e)
                print("error while fetching account data at height {}. Retrying...".format(height))
        accounts.extend(MessageToDict(res)["accounts"])
        if res.pagination.next_key:
            request_msg.pagination.key = res.pagination.next_key
        else:
            next = False
        print("loaded", len(accounts), end="\r")

    return accounts


def get_unlockable_coins(height, addr):
    request_msg = query_lockup.AccountUnlockableCoinsRequest()
    request_msg.owner = addr
    response_msg =query_lockup.AccountUnlockableCoinsResponse
    done = False
    while not done:
        try:
            res = _send_abci_query(request_msg=request_msg,
                                   path="/osmosis.lockup.Query/AccountUnlockableCoins",
                                   response_msg=response_msg,
                                   height=height)
            done = True
        except Exception as e:
            print(e)
            print("error while fetching unlockable coins for address {} at height {}. Retrying...".format(addr, height))
    res = MessageToDict(res)
    return res["coins"] if "coins" in res else []


def get_locked_coins(height, addr):
    request_msg = query_lockup.AccountLockedCoinsRequest()
    request_msg.owner = addr
    response_msg =query_lockup.AccountLockedCoinsResponse
    done = False
    while not done:
        try:
            res = _send_abci_query(request_msg=request_msg,
                                   path="/osmosis.lockup.Query/AccountLockedCoins",
                                   response_msg=response_msg,
                                   height=height)
            done = True
        except Exception as e:
            print(e)
            print("error while fetching locked coins for address {} at height {}. Retrying...".format(addr, height))
    res = MessageToDict(res)
    return res["coins"] if "coins" in res else []


def get_pools_shares(height, addr, pool_ids):
    # returns the list of pool ids and corresponding amounts (in same order)
    # of bonded and unbonded shares owned by the address
    denom_pref = "gamm/pool/{}"
    denoms = [denom_pref.format(pool_id) for pool_id in pool_ids]
    locked = get_locked_coins(height, addr)
    unlockable = get_unlockable_coins(height, addr)
    # `add_coins` defined in cell "define helper functions to handle coins and pool assets", sec. 4
    all_bonded = add_coins(locked, unlockable)
    all_bonded = [c for c in all_bonded if c["denom"] in denoms]
    balances = get_onchain_balances(height, addr)
    balances = [c for c in balances if c["denom"] in denoms]
    bonded = []
    unlocked = []
    for pool_id in pool_ids:
        for c in all_bonded:
            if c["denom"] == denom_pref.format(pool_id):
                bonded.append(c["amount"])
                break
        else:
            bonded.append(None)
        for c in balances:
            if c["denom"] == denom_pref.format(pool_id):
                unlocked.append(c["amount"])
                break
        else:
            unlocked.append(None)
    return pool_ids, unlocked, bonded


### get accounts with LP shares in affected pools

This takes really long time. Load the provided `lp-providers.csv` (see below) preferably.

In [ ]:
# get all accounts
accounts = get_accounts(HALT_HEIGHT)

In [17]:
# use dask for some sweet parallelism
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

# at this point we are just fetching data, so we do it for all pools affected
# in case needed in the future
affected_pools_ids = [1, 2, 497, 561, 573, 584, 597, 637, 641, 678, 690, 704, 712, 722]

remaining = num_accounts = len(accounts)
step = 1000
pbar = ProgressBar()
pbar.register()
lp_providers = []

In [ ]:
while remaining > 0:
    # create pandas dataframe
    start_idx = num_accounts - remaining
    addresses = pd.DataFrame.from_records(accounts[start_idx:start_idx+step])

    # exclude module accounts
    addresses = addresses[addresses["@type"] != "/cosmos.auth.v1beta1.ModuleAccount"]

    # get all addresses
    addresses = addresses.apply(lambda row: row.address if row["@type"] == "/cosmos.auth.v1beta1.BaseAccount" else row.baseVestingAccount["baseAccount"]["address"], axis=1)
    daddresses = dd.from_pandas(addresses, npartitions=32)

    # find LP providers
    lp_providers_step = daddresses.map_partitions(lambda x: x.apply(lambda a: (a, *get_pools_shares(HALT_HEIGHT, a, affected_pools_ids))), meta=(None, 'int64'))
    lp_providers_step = pd.DataFrame.from_records(lp_providers_step.compute())
    lp_providers_step.columns = ["address", "pool_id", "shares_unlocked", "shares_bonded"]

    lp_providers.append(lp_providers_step)
    remaining -= step
    print("remaining:", max(0, remaining))

In [ ]:
pbar.unregister()
lp_providers = pd.concat(lp_providers, axis=0, ignore_index = True, join='outer')

# one row per pool_id
lp_providers = lp_providers.explode(column=["pool_id", "shares_unlocked", "shares_bonded"])

# drop empty rows
lp_providers = lp_providers[(lp_providers["shares_unlocked"].notnull()) | (lp_providers["shares_bonded"].notnull())]
lp_providers

### check if LPs data is consistent with total shares for pools

[optional] in case needed to verify consistency

In [ ]:
maccs = pd.DataFrame.from_records(accounts)

maccs = maccs[maccs["@type"] == "/cosmos.auth.v1beta1.ModuleAccount"]
maccs = maccs.apply(lambda row: row.baseAccount["address"], axis=1)
maccs.reset_index(inplace=True, drop=True)
dmaccs = dd.from_pandas(maccs, npartitions=32)

maccs_shares = dmaccs.map_partitions(lambda x: x.apply(lambda a: (a, *get_pools_shares(HALT_HEIGHT, a, affected_pools_ids))), meta=(None, 'int64'))
maccs_shares = pd.DataFrame.from_records(maccs_shares.compute())
maccs_shares.columns = ["address", "pool_id", "shares_unlocked", "shares_bonded"]
maccs_shares.drop(columns=["shares_bonded"], inplace=True)

maccs_shares = maccs_shares.explode(column=["pool_id", "shares_unlocked"])
maccs_shares = maccs_shares[maccs_shares["shares_unlocked"].notnull()]

In [319]:
for pool_id in affected_pools_ids:
    pool_lp_providers = lp_providers[lp_providers["pool_id"] == pool_id]
    # remove lockup module account
    pool_maccs_shares = maccs_shares[maccs_shares.address != "osmo1njty28rqtpw6n59sjj4esw76enp4mg6g7cwrhc"]
    pool_maccs_shares = pool_maccs_shares[pool_maccs_shares["pool_id"] == pool_id]
    
    # aggretate sum of all bonded + unlocked shares for all LPs
    unlocked = pool_lp_providers.shares_unlocked.apply(lambda v: int(v) if type(v) is float and not math.isnan(v) or type(v) is str else v).sum()
    bonded = pool_lp_providers.shares_bonded.apply(lambda v: int(v) if type(v) is float and not math.isnan(v) or type(v) is str else v).sum()
    total_shares = unlocked + bonded

    maccs_tot_shares = pool_maccs_shares.shares_unlocked.apply(lambda v: int(v) if type(v) is float and not math.isnan(v) or type(v) is str else v).sum()
    if maccs_tot_shares > 0:
        print("pool:", pool_id, "module accounts shares:", maccs_tot_shares, "other accounts shares:", total_shares)
    total_shares += maccs_tot_shares

    if total_shares - int(corrupted_pools[pool_id]["totalShares"]["amount"]) != 0:
        print("mismatch of total computed shares from LP providers (pool: {}): expected {} got {} [{}]".format(pool_id, corrupted_pools[pool_id]["totalShares"]["amount"], total_shares, total_shares - int(corrupted_pools[pool_id]["totalShares"]["amount"])))

    lockup_macc_shares = maccs_shares[(maccs_shares.address == "osmo1njty28rqtpw6n59sjj4esw76enp4mg6g7cwrhc") & (maccs_shares["pool_id"] == pool_id)].shares_unlocked.apply(lambda v: int(v) if type(v) is float and not math.isnan(v) or type(v) is str else v).sum()
    if lockup_macc_shares - bonded!= 0:
        print("mismatch of bonded shares (pool: {}): expected {} got {} [{}]".format(pool_id, lockup_macc_shares, bonded, bonded - lockup_macc_shares))

pool: 1 module accounts shares: 1253949019890991387 other accounts shares: 392330249879706319995300547
pool: 2 module accounts shares: 6076188108496233 other accounts shares: 557461895925129150225766035


### save LP providers to `lp-providers.csv`

In [294]:
lp_providers.to_csv("csv/lp-providers.csv", index=None)

### reload from `lp-providers.csv` if re-starting from here

load csv to avoid having to run the previous cells

In [320]:
lp_providers = pd.read_csv("csv/lp-providers.csv")

### calculate Rektdrop amounts

Use the total shares owned (discounting illegal shares, if possessed) to simulate a complete exit on both `proper_pools` (with fixed liquidity and shares, as per section 4) and `corrupted_pools` (onchain history). The difference will be the amount that the account should be airdropped to compensate for the exploit.

While doing this analysis, we also keep track of whether the account previously stole funds or is in possession of illegal shares (that entitle the account to claim more liquidity than deposited). We discount these amounts from the airdrop, and in case the result is negative the address is excluded altogether.

In [342]:
# `burn_shares`, `add_coins` and `sub_coins` are defined in sec. 4

affected_pools_ids = [1, 678, 704, 712] # only on considered pools

rektdrop = []
excluded = []

for pool_id in affected_pools_ids:
    pool_lp_providers = lp_providers[lp_providers["pool_id"] == pool_id]
    addrs = []

    for _, lp_provider in pool_lp_providers.iterrows():
        addr = lp_provider.address
        unlocked = lp_provider.shares_unlocked or 0
        unlocked = 0 if type(unlocked) is float and math.isnan(unlocked) else int(unlocked)
        bonded = lp_provider.shares_bonded or 0
        bonded = 0 if type(bonded) is float and math.isnan(bonded) else int(bonded)
        total_shares = unlocked + bonded
        airdrop = []
        illegal_liquidity = [] # can either be stolen liquidity or liquidity claimable with inflated shares possessed
        inflated_shares = 0

        # if address is in `affected_wallets` retrieve relevant data
        if addr in affected_wallets and pool_id in affected_wallets[addr]:
            if "shares_inflated" in affected_wallets[addr][pool_id]:
                inflated_shares += affected_wallets[addr][pool_id]["shares_inflated"]
            if "shares_inflated_locked" in affected_wallets[addr][pool_id]:
                inflated_shares += affected_wallets[addr][pool_id]["shares_inflated_locked"]
            if "tokens_owed" in affected_wallets[addr][pool_id]:
                airdrop = add_coins(airdrop, affected_wallets[addr][pool_id]["tokens_owed"])
            if "tokens_stolen" in affected_wallets[addr][pool_id]:
                illegal_liquidity = add_coins(illegal_liquidity, affected_wallets[addr][pool_id]["tokens_stolen"])
        
        claim_shares = total_shares - inflated_shares
        if claim_shares < 0:
            raise Exception("should not happen")
        exit_amt = burn_shares(corrupted_pools[pool_id], claim_shares)
        proper_exit_amt = burn_shares(proper_pools[pool_id], claim_shares)

        if inflated_shares > 0:
            # add the liquidity that could be obtained by claiming the inflated shares to `illegal_liquidity`
            illegal_liquidity = add_coins(illegal_liquidity, burn_shares(corrupted_pools[pool_id], inflated_shares))

        diff = sub_coins(proper_exit_amt, exit_amt)
        if diff:
            airdrop = add_coins(airdrop, diff)
            if illegal_liquidity:
                # only airdrop to addres whatever remains after removing `illegal_liquidity` (penalization)
                # if the sum of liquidity stolen or claimable with inflated shares is bigger than the airdrop
                # amount, then account is excluded from rektdrop
                try:
                    diff = sub_coins(airdrop, illegal_liquidity)
                    rektdrop.append([addr, pool_id, diff])
                except ArithmeticError as e:
                    print("[warn] excluding addr {} on pool {}".format(addr, pool_id))
                    excluded.append([addr, pool_id, airdrop, illegal_liquidity])
            else:
                rektdrop.append([addr, pool_id, airdrop])
        addrs.append(addr)

    for addr, pools in affected_wallets.items():
        # dig out possibly remaining addresses that did not have any LP share at halt but had a refund due
        if pool_id in pools and "tokens_owed" in pools[pool_id] and addr not in addrs:
            rektdrop.append([addr, pool_id, pools[pool_id]["tokens_owed"]])

[warn] excluding addr osmo10rgny3xd5fvg02wnmy4h9zuktnfthvrlwdjwf5 on pool 1
[warn] excluding addr osmo12098ugn5gac6qnhcys065dxa29g6szdpe2zvyn on pool 1
[warn] excluding addr osmo127cz3qd25qefq2d3l8zdl9xqzatskjcq4n5eg2 on pool 1
[warn] excluding addr osmo12asyaccxrau33q8l780cgzkyzydqngfvla7deg on pool 1
[warn] excluding addr osmo13j44mhvhafkpxqjfz6zrknverjc67jcc8gj7qr on pool 1
[warn] excluding addr osmo13t4fs279dq0ne2lk7juyxfhfsnz9gsfw73f5tu on pool 1
[warn] excluding addr osmo14pakf28w2vmeemfcfhlt3xgpah24548lzdp5c8 on pool 1
[warn] excluding addr osmo14vae3yzzm43j4uc6x82e0ltxaxwjlu2szdm2tv on pool 1
[warn] excluding addr osmo158nm2fsv70dq74ed8e3ccs6e44c3e65kqmxfut on pool 1
[warn] excluding addr osmo15qqsnsa2yqweqasnahke6vcgc0pgd692295cdz on pool 1
[warn] excluding addr osmo15u4c3rpyy22ne9tv46jxh82c4cu07varm8zvu3 on pool 1
[warn] excluding addr osmo15wk3htq4p024tfc6nagvgusra0tlt3hhv2sw9d on pool 1
[warn] excluding addr osmo166cfn9s7qa0klrj7t0ru35pp34yxvqdlhsqlfy on pool 1
[warn] exclu

## 8. Save Rektdrop results to `rektdrop.csv`

In [353]:
rektdrop_df = pd.DataFrame.from_records(rektdrop)
rektdrop_df.columns = ["address", "pool_id", "amount"]
rektdrop_df.amount = rektdrop_df.amount.apply(token_to_str) # token_to_str defined in first cell of sec. 5
rektdrop_df.sort_values(by=["address", "pool_id"], ascending=True, inplace=True)

rektdrop_df

,address,pool_id,amount
0,osmo10009zx2uyaw2zkj7ye5zmmxsnu2d9dulh6fd8d,1,81ibc/27394FB092D2ECCD56123C74F36E4C1F926001CE...
109363,osmo1000xz25ydz8h9rwgnv30l9p0x500dvj0wv50ft,678,2942392ibc/D189335C6E4A68B513C10AB227BF1C1D38C...
1,osmo100266xcz9cwdufdcqu0vm4wha4zxzgqelcv8jn,1,2117614ibc/27394FB092D2ECCD56123C74F36E4C1F926...
2,osmo10028te5z5rdfh9xhu9nmmqcdxdqxs3df7qdfuu,1,61194ibc/27394FB092D2ECCD56123C74F36E4C1F92600...
3,osmo10035cturpm08z734pwlphm4fav8h0jempg0ufx,1,29ibc/27394FB092D2ECCD56123C74F36E4C1F926001CE...
...,...,...,...
109306,osmo1zzypzvvavvck8nmtupclh4dn5jwydykpslvjnn,1,230248ibc/27394FB092D2ECCD56123C74F36E4C1F9260...
109307,osmo1zzz6n6kxawm6qepu6266r989tmsmpmcgccd2fm,1,209249ibc/27394FB092D2ECCD56123C74F36E4C1F9260...
109308,osmo1zzzgswmf6kpvvju4vz4fm6ltvtcev00f8wm9ev,1,68517ibc/27394FB092D2ECCD56123C74F36E4C1F92600...
109309,osmo1zzzk2244camjzltt9uau9u2xh4y7705hf4duex,1,100182ibc/27394FB092D2ECCD56123C74F36E4C1F9260...


In [354]:
excluded_df = pd.DataFrame.from_records(excluded)
excluded_df.columns = ["address", "pool_id", "airdrop_amount", "debt_total"]
excluded_df.airdrop_amount = excluded_df.airdrop_amount.apply(token_to_str)
excluded_df.debt_total = excluded_df.debt_total.apply(token_to_str)
excluded_df.sort_values(by=["address", "pool_id"], ascending=True, inplace=True)

excluded_df

,address,pool_id,airdrop_amount,debt_total
152,osmo103w2uwpdf9p708c83nkyd0ys6fgqt94tdvjtgn,712,104163ibc/D1542AA8762DB13087D8364F3EA6509FD6F0...,849822ibc/D1542AA8762DB13087D8364F3EA6509FD6F0...
72,osmo109aaa6tftveephmkppldkgjlj70t6cdx2gcx88,678,39330ibc/D189335C6E4A68B513C10AB227BF1C1D38C74...,265456ibc/D189335C6E4A68B513C10AB227BF1C1D38C7...
73,osmo10dc7lataqm62tze9tm47f8sesrws709zfd3xhh,678,142168ibc/D189335C6E4A68B513C10AB227BF1C1D38C7...,208361ibc/D189335C6E4A68B513C10AB227BF1C1D38C7...
74,osmo10qx8j604ll0tkzvel704z97hukwsmfmy3ttyf9,678,2039ibc/D189335C6E4A68B513C10AB227BF1C1D38C746...,13760ibc/D189335C6E4A68B513C10AB227BF1C1D38C74...
0,osmo10rgny3xd5fvg02wnmy4h9zuktnfthvrlwdjwf5,1,116949ibc/27394FB092D2ECCD56123C74F36E4C1F9260...,4667423ibc/27394FB092D2ECCD56123C74F36E4C1F926...
...,...,...,...,...
120,osmo1z7qd2j77knchudz3a98r2e0dkquhx3r3pzslt7,678,4714332ibc/D189335C6E4A68B513C10AB227BF1C1D38C...,15231685ibc/D189335C6E4A68B513C10AB227BF1C1D38...
121,osmo1z8ezyzy9rx5pswhxgkca38pewr25nw53wdmr9q,678,17709ibc/D189335C6E4A68B513C10AB227BF1C1D38C74...,43620ibc/D189335C6E4A68B513C10AB227BF1C1D38C74...
151,osmo1za34x4r5fq8jmuk6gvsufx5pt5vz2hltgtxnpz,704,594858014657242ibc/EA1D43981D5C9A1C4AAEA9C23BB...,582747019179374ibc/EA1D43981D5C9A1C4AAEA9C23BB...
122,osmo1zepwhr77p406jnz44xt007nfxat9ls95z3cgnv,678,7448987ibc/D189335C6E4A68B513C10AB227BF1C1D38C...,26827165ibc/D189335C6E4A68B513C10AB227BF1C1D38...


In [355]:
rektdrop_df.to_csv("csv/rektdrop.csv", index=None)
excluded_df.to_csv("csv/rektdrop-excluded.csv", index=None)

## 9. APPENDIX

### transactions occurrences

In [304]:
df.groupby("@type").size().reset_index(name="number_of_txs")

,@type,number_of_txs
0,/osmosis.gamm.v1beta1.MsgExitPool,832
1,/osmosis.gamm.v1beta1.MsgJoinPool,1972
2,/osmosis.gamm.v1beta1.MsgJoinSwapExternAmountIn,2018
3,/osmosis.gamm.v1beta1.MsgSwapExactAmountIn,39187
4,/osmosis.gamm.v1beta1.MsgSwapExactAmountOut,455
5,/osmosis.lockup.MsgLockTokens,3228
6,/osmosis.superfluid.MsgLockAndSuperfluidDelegate,260
7,/osmosis.superfluid.MsgUnPoolWhitelistedPool,19


### group transactions by sender and sort by number of txs

In [305]:
(df.groupby(["@type", "sender"])
    .size()
    .reset_index(name="number_of_txs")
    .sort_values(by=["number_of_txs"], ascending=False))

,@type,sender,number_of_txs
7263,/osmosis.gamm.v1beta1.MsgSwapExactAmountIn,osmo1lyncm90mfw9fp9xpdnxgu6sjkh8egnzsqm3l5m,2727
6393,/osmosis.gamm.v1beta1.MsgSwapExactAmountIn,osmo1h9ac2c382h4hyadx3mlqsgc8wus53dn67kl9p9,1780
5864,/osmosis.gamm.v1beta1.MsgSwapExactAmountIn,osmo1f6k7kcs2jcfpkyr86vw3hdqtz6775ekfahjm7u,1526
8086,/osmosis.gamm.v1beta1.MsgSwapExactAmountIn,osmo1qmnw9h2fd0h2sqsq2h937n0c4kjz4jlqf3x9w4,1487
3964,/osmosis.gamm.v1beta1.MsgSwapExactAmountIn,osmo15vdjje8009ly9rudapxjcht5h9vshc7hw3a7zy,1044
...,...,...,...
4392,/osmosis.gamm.v1beta1.MsgSwapExactAmountIn,osmo17newevzrcmwfu89n4w8fu0ywyn5fs8dqm3vhs4,1
4389,/osmosis.gamm.v1beta1.MsgSwapExactAmountIn,osmo17ms3sn7qe48xtdnl5mzfk8u3gwhln6hhlr36lp,1
4388,/osmosis.gamm.v1beta1.MsgSwapExactAmountIn,osmo17mmpk8m6s57k60e357en52gens88eukmnec20s,1
4387,/osmosis.gamm.v1beta1.MsgSwapExactAmountIn,osmo17mmhl3d3rsmc7457gw30p20qgj3d36mgcefprd,1


### find wallets-by-pools that did either join or exit. Count occurrences of joins/exits per pool

In [306]:
exit_joins_df = (df[(df["@type"].str.contains("MsgJoinPool")) | (df["@type"].str.contains("MsgExitPool"))]
    .groupby(["@type", "sender", "poolId"])
    .size()
    .reset_index(name="number_of_txs")
    .sort_values(by=["number_of_txs"], ascending=False))
exit_joins_df = exit_joins_df[exit_joins_df["@type"].str.contains("MsgJoinPool")].merge(exit_joins_df[exit_joins_df["@type"].str.contains("MsgExitPool")], on=["sender", "poolId"], how="outer")
exit_joins_df.drop(columns=["@type_x", "@type_y"], inplace=True)
exit_joins_df.columns = ["address", "poolId", "joins", "exits"]

exit_joins_df.joins = exit_joins_df.joins.apply(lambda v: 0 if math.isnan(v) else int(v))
exit_joins_df.exits = exit_joins_df.exits.apply(lambda v: 0 if math.isnan(v) else int(v))

exit_joins_df

,address,poolId,joins,exits
0,osmo18qx59wy8s3ytax3e0akna934e86mw776vlzjtq,1,45,47
1,osmo1yglld3aary7lnrrn2dz7la84kmnmen4kpsxzay,712,43,42
2,osmo1hq8tlgq0kqz9e56532zghdhz7g8gtjymdltqer,678,28,28
3,osmo1jfxcl8ja3nnfjduqemptknz2j6nk6502zp3rte,561,25,25
4,osmo18qx59wy8s3ytax3e0akna934e86mw776vlzjtq,712,19,17
...,...,...,...,...
2139,osmo1cww8ylq7ge7ps8776qvugasd6yks3c7u6wqyht,561,0,1
2140,osmo1cwux6ehfyg98zj7x3vgr7gz33l2kye3n42su5t,1,0,1
2141,osmo1cwj3vqxk6p7e6ntxt8925m3l9d6l0jr554d3e7,1,0,1
2142,osmo1crykcdaxm3u4flnjxq0kmyye7uqy9tcesac6u2,651,0,1


### find wallets that used the exploit on more than one pool (possibly moving stolen liquidity across pools)

In [564]:
num_pools_exploited = exit_joins_df[(exit_joins_df.joins > 0) & (exit_joins_df.exits > 0)].groupby("address").size().reset_index(name="number_of_pools")

num_pools_exploited[num_pools_exploited.number_of_pools > 1]

,address,number_of_pools
4,osmo18qx59wy8s3ytax3e0akna934e86mw776vlzjtq,3
8,osmo1cw73elmtjvzg4gjdy9yv7hu6nnfl8m7z4ah5wl,2
12,osmo1jdh7eeyaar0tyask0r6w2228uh5wrd0pxtcfwr,4
16,osmo1tg70tuzekpd376dpqr68yx5a7r709w6x8jtxha,3


In [572]:
exit_joins_df[exit_joins_df.address.isin(num_pools_exploited[num_pools_exploited.number_of_pools > 1].address)]

,address,poolId,joins,exits
0,osmo18qx59wy8s3ytax3e0akna934e86mw776vlzjtq,1,45,47
4,osmo18qx59wy8s3ytax3e0akna934e86mw776vlzjtq,712,19,17
5,osmo1jdh7eeyaar0tyask0r6w2228uh5wrd0pxtcfwr,678,17,17
7,osmo1tg70tuzekpd376dpqr68yx5a7r709w6x8jtxha,1,16,16
9,osmo18qx59wy8s3ytax3e0akna934e86mw776vlzjtq,704,13,13
58,osmo1cw73elmtjvzg4gjdy9yv7hu6nnfl8m7z4ah5wl,722,2,1
59,osmo1cw73elmtjvzg4gjdy9yv7hu6nnfl8m7z4ah5wl,1,2,1
216,osmo1jdh7eeyaar0tyask0r6w2228uh5wrd0pxtcfwr,1,1,1
304,osmo1jdh7eeyaar0tyask0r6w2228uh5wrd0pxtcfwr,704,1,1
307,osmo1jdh7eeyaar0tyask0r6w2228uh5wrd0pxtcfwr,2,1,1


address `osmo1cw73elmtjvzg4gjdy9yv7hu6nnfl8m7z4ah5wl` uses extremely low amounts (less than 1 OSMO) for all the txs, so we can ignore it

The remaining addresses even if moving stolen liquidity across pools end up evening out joins/exits (`osmo18qx59wy8s3ytax3e0akna934e86mw776vlzjtq` does 2 exits for 1/2 shares obtained via a previous join, and a `MsgJoinSwapExternAmountIn`). Hence there's no stolen liquidity in any pool at `HALT_HEIGHT` that should be discounted to adjust the pools balance.

### diff of joins/exits amounts

quick examples of how one could compare results

#### addr `osmo18qx59wy8s3ytax3e0akna934e86mw776vlzjtq` on pool `1`

In [532]:
exits_pool1_osmo18qx5 = df[(df.sender == "osmo18qx59wy8s3ytax3e0akna934e86mw776vlzjtq") & (df["@type"] == "/osmosis.gamm.v1beta1.MsgExitPool") & (df.poolId == 1)].tokensOut
joins_pool1_osmo18qx5 = df[(df.sender == "osmo18qx59wy8s3ytax3e0akna934e86mw776vlzjtq") & (df["@type"] == "/osmosis.gamm.v1beta1.MsgJoinPool") & (df.poolId == 1)].tokensIn
joinsSwap_pool1_osmo18qx5 = df[(df.sender == "osmo18qx59wy8s3ytax3e0akna934e86mw776vlzjtq") & (df["@type"] == "/osmosis.gamm.v1beta1.MsgJoinSwapExternAmountIn") & (df.poolId == 1)].tokensIn

tot_exits = []
for _, e in exits_pool1_osmo18qx5.iteritems():
    tot_exits = add_coins(tot_exits, [parse_token_str(t) for t in e.split(",")])

tot_joins = []
for _, j in joins_pool1_osmo18qx5.iteritems():
    tot_joins = add_coins(tot_joins, [parse_token_str(t) for t in j.split(",")])
for _, j in joinsSwap_pool1_osmo18qx5.iteritems():
    tot_joins = add_coins(tot_joins, [parse_token_str(t) for t in j.split(",")])

# from .csv: 155854901139ibc/27394FB092D2ECCD56123C74F36E4C1F926001CEADA9CA97EA622B25F41E5EB2,1218849678601uosmo
print(token_to_str(sub_coins(tot_exits, tot_joins)))

155854126701ibc/27394FB092D2ECCD56123C74F36E4C1F926001CEADA9CA97EA622B25F41E5EB2,1218855862201uosmo


#### addr `osmo1hq8tlgq0kqz9e56532zghdhz7g8gtjymdltqer` on pool `678`

In [543]:
exits_pool678_osmo1hq8t = df[(df.sender == "osmo1hq8tlgq0kqz9e56532zghdhz7g8gtjymdltqer") & (df["@type"] == "/osmosis.gamm.v1beta1.MsgExitPool") & (df.poolId == 678)].tokensOut
joins_pool678_osmo1hq8t = df[(df.sender == "osmo1hq8tlgq0kqz9e56532zghdhz7g8gtjymdltqer") & (df["@type"] == "/osmosis.gamm.v1beta1.MsgJoinPool") & (df.poolId == 678)].tokensIn

tot_exits = []
for _, e in exits_pool678_osmo1hq8t.iteritems():
    tot_exits = add_coins(tot_exits, [parse_token_str(t) for t in e.split(",")])

tot_joins = []
for _, j in joins_pool678_osmo1hq8t.iteritems():
    tot_joins = add_coins(tot_joins, [parse_token_str(t) for t in j.split(",")])

# from .csv: 790892713947uosmo,900211662047ibc/D189335C6E4A68B513C10AB227BF1C1D38C746766278BA3EEB4FB14124F1D858
print(token_to_str(sub_coins(tot_exits, tot_joins)))

900019807748ibc/D189335C6E4A68B513C10AB227BF1C1D38C746766278BA3EEB4FB14124F1D858,791060071158uosmo


#### addr `osmo1yglld3aary7lnrrn2dz7la84kmnmen4kpsxzay` on pool `712`

In [544]:
exits_pool712_osmo1ygll = df[(df.sender == "osmo1yglld3aary7lnrrn2dz7la84kmnmen4kpsxzay") & (df["@type"] == "/osmosis.gamm.v1beta1.MsgExitPool") & (df.poolId == 712)].tokensOut
joins_pool712_osmo1ygll = df[(df.sender == "osmo1yglld3aary7lnrrn2dz7la84kmnmen4kpsxzay") & (df["@type"] == "/osmosis.gamm.v1beta1.MsgJoinPool") & (df.poolId == 712)].tokensIn

tot_exits = []
for _, e in exits_pool712_osmo1ygll.iteritems():
    tot_exits = add_coins(tot_exits, [parse_token_str(t) for t in e.split(",")])

tot_joins = []
for _, j in joins_pool712_osmo1ygll.iteritems():
    tot_joins = add_coins(tot_joins, [parse_token_str(t) for t in j.split(",")])

# from .csv: 149148498ibc/D1542AA8762DB13087D8364F3EA6509FD6F009A34F00426AF9E4F9FA85CBBF1F,40001133062uosmo plus 580397812576743768 gamm shares locked
print(token_to_str(sub_coins(tot_exits, tot_joins)))

149113069ibc/D1542AA8762DB13087D8364F3EA6509FD6F009A34F00426AF9E4F9FA85CBBF1F,39993508750uosmo


#### addr `osmo1jfxcl8ja3nnfjduqemptknz2j6nk6502zp3rte` on pool `561`

In [560]:
exits_pool561_osmo1jfxc = df[(df.sender == "osmo1jfxcl8ja3nnfjduqemptknz2j6nk6502zp3rte") & (df["@type"] == "/osmosis.gamm.v1beta1.MsgExitPool") & (df.poolId == 561)].tokensOut
joins_pool561_osmo1jfxc = df[(df.sender == "osmo1jfxcl8ja3nnfjduqemptknz2j6nk6502zp3rte") & (df["@type"] == "/osmosis.gamm.v1beta1.MsgJoinPool") & (df.poolId == 561)].tokensIn

tot_exits = []
for _, e in exits_pool561_osmo1jfxc.iteritems():
    tot_exits = add_coins(tot_exits, [parse_token_str(t) for t in e.split(",")])

tot_joins = []
for _, j in joins_pool561_osmo1jfxc.iteritems():
    tot_joins = add_coins(tot_joins, [parse_token_str(t) for t in j.split(",")])

# from .csv: 248712074259ibc/0EF15DF2F02480ADE0BB6E85D9EBB5DAEA2836D3860E9F97F9AADE4F57A31AA0 plus 777274357520858448297301 shares to return and burn
# account had most of the stolen assets added to the pool at halt
print(token_to_str(tot_exits), token_to_str(tot_joins))

# first join is 16194410932ulunc, 676941uosmo (tx: DC820F6A14FA37349B09442CD6DC414F4F31B480B134741B148BAB7CEFD0DB01)
print(545483866425767 - 545221122485296 - 248712074259, "ulunc") #ulunc
print(23644739710 - 23644063102, "uosmo") #uosmo

545483866425767ibc/0EF15DF2F02480ADE0BB6E85D9EBB5DAEA2836D3860E9F97F9AADE4F57A31AA0,23644063102uosmo 545221122485296ibc/0EF15DF2F02480ADE0BB6E85D9EBB5DAEA2836D3860E9F97F9AADE4F57A31AA0,23644739710uosmo
14031866212 ulunc
676608 uosmo
